In [25]:
import numpy as np
import re
from sklearn import linear_model
from sklearn.svm import SVC
from coreNlpUtil import *
subject_word_to_indx = { "ship":2,"boy":4 }
action_word_to_indx = {"sail" : 3 }
object_word_to_indx = {"sea" : 5 }
vocab_count = 6
partial_captions = [(2,3,5)]
cap = np.array([0,1,2,3,4]).reshape((5,1))
images_features = np.arange(5000).reshape((5,1000))

In [2]:

n_samples = len(images_features)

X_train = images_features
y_train = cap

logistic = linear_model.LogisticRegression()

print('Logistic Regression score: %f'
      % logistic.fit(X_train, y_train).score(X_train, y_train))
logistic.predict(X_train)

Logistic Regression score: 1.000000


/Users/danielsampetethiyagu/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 2, 3, 4])

In [3]:
import pickle
d = pickle.load(open('flickr8k_Entire_dataset_with_cnn_features.pkl'))

In [4]:
for i in (d[:10]):
    print i['sentences'][0]['raw']


A black dog is running after a white dog in the snow .
A little baby plays croquet .
A brown dog in the snow has something hot pink in its mouth .
A brown dog is running along a beach .
A black and white dog with a red Frisbee standing on a sandy beach .
A cyclist wearing a red helmet is riding on the pavement .
A man dressed in a purple shirt and red bandanna smiles at the people watching him .
A boy wearing a red t-shirt is running through woodland .
A girl in a white dress .
A skier in a yellow jacket is airborne above the mountains .


In [ ]:
"dog"
"run"
"snow"

"baby" 
"play"
"truck"

"dog"
"hold"
"snow"


"dog" 
"run"
"beach"


"dog"
"stand"
"beach"

"cyclist"
"ride"
"pavement"


A man dressed in a purple shirt and red bandanna smiles at the people watching him .
A boy wearing a red t-shirt is running through woodland .
A girl in a white dress .
A skier in a yellow jacket is airborne above the mountains .

In [5]:
subj_word_dict = {"dog":1,"baby":2,"cyclist":3}
action_word_dict = {"run":1,"play":2, "hold":3, "stand":4, "ride":5}
area_word_dict = {"snow":1, "truck":2, "beach": 3, "pavement":4}
y_sub = np.array([1,2,1,1,1,3])
y_action = np.array([1,2,3,1,4,5])
y_area = np.array([1,2,1,3,3,4])

In [6]:
X_train = []
for i in (d[:6]):
    X_train.append(i['cnn features'])
X_train = np.array(X_train)

In [7]:
logistic_subject = linear_model.LogisticRegression()
logistic_action = linear_model.LogisticRegression()
logistic_area = linear_model.LogisticRegression()

print('Logistic Regression score: %f'
      % logistic_subject.fit(X_train, y_sub).score(X_train, y_sub))

print('Logistic Regression score: %f'
      % logistic_action.fit(X_train, y_action).score(X_train, y_action))

print('Logistic Regression score: %f'
      % logistic_area.fit(X_train, y_area).score(X_train, y_area))

Logistic Regression score: 0.666667
Logistic Regression score: 0.666667
Logistic Regression score: 0.666667


In [8]:
logistic_action.predict(X_train)

array([1, 1, 3, 1, 4, 1])

In [9]:
import nltk

def make_from_prep(entire_parse):
    for parse in entire_parse:
        m = re.search("prep_(.*)", parse[0])
        #print "CAME IN "
        if(m):
            return process_text((parse[1],m.group(1),parse[2]))
    return None

def combine_nn_dobj(nn, dobj):
    return process_text((nn[0][0], dobj[0][1], dobj[0][0]))

def comb_acomp_subj(subj,acomp):
    return process_text((subj[0][0], subj[0][1], acomp[0][0]))

def remove_char_dep_res(text):
    return re.sub("{.*}","",text)

def process_text(triplet):
    return re.sub("{.*}","",triplet[0]), re.sub("{.*}","",triplet[1]), re.sub("{.*}","",triplet[2])
    #triplet[1] = 
    #triplet[2] = re.sub("{.*}","",triplet[2])
    #return triplet

def find_triplet_from_prep(entire_parse):
    for parse in entire_parse:
        m = re.search("prep_(.*)", parse[0])
        #print "CAME IN "
        if(m):
            word = remove_char_dep_res(parse[1])
            #print word, "CHECKING FOR NN",nltk.pos_tag([word])
            if(nltk.pos_tag([word])[0][1] == "NN"):
                #print "CAME IN *********"
                m = re.search("prep_(.*)", parse[0])
                return process_text((parse[1], m.group(1), parse[2]))
    return None
    
def find_category_pos(pos):
    if(pos in ["VBG"]):
        return "Verb"
    if(pos in ["NN"]):
        return "Noun"
    return "UNKNOWN Category"

def find_triplet_nltk(root, prep_parses, dep_parse):
    Verb = []
    Noun = []
    if len(root) == 0:
        return None
    for prep_parse in prep_parses:
        if(root == prep_parse[1]):
            word = (remove_char_dep_res(prep_parse[0]))
            #print word
            #print find_category_pos(nltk.pos_tag([word])[0][1])
            if(find_category_pos(nltk.pos_tag([word])[0][1]) == "Verb"):
                
                Verb.append(word)
            if(find_category_pos(nltk.pos_tag([word])[0][1]) == "Noun"):
                Noun.append(word)
            if(len(Verb) >= 1 and len(Noun) >= 1):
                break;
    if(len(Verb) >= 1 and len(Noun) >= 1):
        return (remove_char_dep_res(root), Verb[0], Noun[0])
    for parse in dep_parse:
        m = re.search("prep_(.*)", parse[0])
        if(m):
            return process_text((root, m.group(1), parse[2]))
        
    return None   

def triplet(dep_parse):
    nsubj = []
    dobj = []
    prep = []
    root = ""
    nn = []
    acomp = []
    prt = []
    advmod = []
    aux = []
    for parse in dep_parse:
        if  parse[0] == "nsubj":
            nsubj.append( (parse[2], parse[1]) )

        elif  parse[0] == "dobj":
            dobj.append( (parse[2], parse[1]) )

        elif re.search("prep_", parse[0]):
            prep.append((parse[2], parse[1]))

        elif parse[0] == "root":
            root = parse[2]
        
        elif  parse[0] == "nn":
            nn.append( (parse[2], parse[1]) )
        elif  parse[0] == "acomp":
            acomp.append( (parse[2], parse[1]) )
            
        elif  parse[0] == "prt":
            prt.append( (parse[2], parse[1]) )
        
        elif  parse[0] == "advmod":
            prt.append( (parse[2], parse[1]) )
        elif  parse[0] == "aux":
            prt.append( (parse[2], parse[1]) )
            
            
    found = False
    if(len(nsubj)>0):
        for nsubj_dep in nsubj:
            for dobj_dep in dobj:
                if(nsubj_dep[1] == dobj_dep[1]):
                    triplet = process_text((nsubj_dep[0], nsubj_dep[1], dobj_dep[0]))
                    print triplet
                    found = True
                    break;
            if(found):
                break;
            
            for prep_dep in prep:
                if(nsubj_dep[1] == prep_dep[1]):
                    triplet = process_text((nsubj_dep[0], nsubj_dep[1], prep_dep[0]))
                    print triplet
                    found = True
                    break;
            if(found):
                break;
    
    elif(len(dobj)>0 and len(nn)>0):
        trip = combine_nn_dobj(nn,dobj)
        print trip
        if(trip!=None):
            triplet = trip
            found = True

    else:
        if(len(root)>0):
            trip = find_triplet_nltk(root, prep, dep_parse)
            if(trip!= None):
                triplet = trip
                print triplet
                found = True
    if(found==False):        
        if(len(nsubj)>0 and len(acomp)> 0):
            triplet = comb_acomp_subj(nsubj,acomp)
            print triplet
            found = True
        elif(len(prep) > 0):
            triplet = find_triplet_from_prep(dep_parse)
            if(triplet!=None):
                print triplet
                found = True
        elif(len(nsubj)>0 and (len(prt)> 0 or len(advmod)>0 or len(aux)>0)):
            for subj_i in nsubj:
                for prt_j in prt:
                    if(subj_i[1]==prt_j[1]):
                        triplet = process_text((subj_i[0],subj_i[1],prt_j[0]))
                        print triplet
                        found = True
                        break;
                if(found==True):
                    break;
                    
                for adv_j in advmod:
                    if(subj_i[1]==adv_j[1]):
                        triplet = process_text((subj_i[0],subj_i[1],adv_j[0]))
                        print triplet
                        found = True
                        break;
                if(found==True):
                    break;
                    
                for aux_j in aux:
                    if(subj_i[1]==aux_j[1]):
                        print (subj_i[0],aux_j[0],subj_i[1])
                        triplet = process_text((subj_i[0],aux_j[0],subj_i[1]))
                        print triplet
                        found = True
                        break;
                if(found==True):
                    break;
        elif(len(nn)>0):
            for nn_item in nn:
                #print dep_parse
                for parse in dep_parse:
                    #print parse[1]
                    if(parse[1] == nn_item[1]):
                        triplet = process_text((nn_item[0],nn_item[1],parse[2]))
                        print triplet
                        found = True
                        break;
                if(found):
                    break
        
        if(not found and len(dobj)>0):
            if(len(root) > 0):
                root_word = remove_char_dep_res(root)
                if nltk.pos_tag([root_word])[0][1] == "NN":
                    #print "CAME IN *****"
                    triplet = process_text((root_word, dobj[0][1], dobj[0][0]))
                    found = True
                    print triplet
#             for dobj_item in dobj:
#                 for parse in dep_parse:
#                     if(parse)
        if(not found and len(prep)>0):
            triplet = make_from_prep(dep_parse)
            if(triplet!=None):
                found = True
                print triplet
        

    if(found == False):
        print "SUBJECTS are "
        print nsubj
        print "OBJECTS are "
        print dobj
        print "PREP are "
        print prep
        print "Entire Dep Parse"
        print dep_parse
        return None
    else:
        return triplet


https://github.com/ma-sultan/monolingual-word-aligner
https://github.com/dasmith/stanford-corenlp-python
Follow above instructions in the monolingual word aligner including the one which states to modify the certain line number 100, THen RUN the Corenlp server and check

In [10]:
def process_dataset(dataset):
    Triplets = []
    X_train = []
    f = open('t.csv', 'w')
    for i in (dataset):
        #print i['sentences'][0]['raw']
        sentence1ParseResult = parseText(i['sentences'][0]['raw'])
        result = dependencyParseAndPutOffsets(sentence1ParseResult)
        triplet_value = triplet(result)
        if(triplet_value!=None):
            f.write("%s, %s, %s, %s\n" %(i['sentences'][0]['raw'], triplet_value[0], triplet_value[1], triplet_value[2]))
            Triplets.append(triplet_value)
            X_train.append(i['cnn features'])    
        #print result
        print ""
    f.close()
    return np.array(X_train), Triplets

In [11]:
def process_triplet_dictionary(allTriplets):
    subj_vocab = []
    action_vocab = []
    object_vocab = []
    for triplet in allTriplets:
        subj_vocab.append(triplet[0])
        action_vocab.append(triplet[1])
        object_vocab.append(triplet[2])
    subj_vocab.append("UNK")
    action_vocab.append("UNK")
    object_vocab.append("UNK")
    subj_word_to_index = {w: i for i, w in enumerate(subj_vocab)}
    subj_index_to_word = {i: w for i, w in enumerate(subj_vocab)}
    action_word_to_index = {w: i for i, w in enumerate(action_vocab)}
    action_index_to_word = {i: w for i, w in enumerate(action_vocab)}
    object_word_to_index = {w: i for i, w in enumerate(object_vocab)}
    object_index_to_word = {i: w for i, w in enumerate(object_vocab)}
    #print subj_word_to_index,subj_index_to_word,action_word_to_index,action_index_to_word,object_word_to_index,object_index_to_word
    return subj_word_to_index,subj_index_to_word,action_word_to_index,action_index_to_word,object_word_to_index,object_index_to_word

In [12]:
def get_y_values(allTriplets,subj_word_to_index,action_word_to_index,object_word_to_index):
    y_subj = []
    y_action = []
    y_obj = []
    for triplet in allTriplets:
        if(triplet[0] in subj_word_to_index):
            y_subj.append(subj_word_to_index[triplet[0]])
        else:
            y_subj.append(subj_word_to_index["UNK"])
        if(triplet[1] in action_word_to_index):
            y_action.append(action_word_to_index[triplet[1]])
        else:
            y_action.append(action_word_to_index["UNK"])
        if(triplet[2] in object_word_to_index):
            y_obj.append(object_word_to_index[triplet[2]])
        else:
            y_obj.append(object_word_to_index["UNK"])
    return np.array(y_subj), np.array(y_action), np.array(y_obj)

def print_output(y_sub, y_action, y_obj, subj_index_to_word, action_index_to_word, object_index_to_word):
    for i in range(len(y_sub)):
        print subj_index_to_word[y_sub[i]], action_index_to_word[y_action[i]], object_index_to_word[y_obj[i]]

def train_model(X_train, y_sub, y_action, y_area):
    logistic_subject = linear_model.LogisticRegression()
    logistic_action = linear_model.LogisticRegression()
    logistic_area = linear_model.LogisticRegression()

    print('Logistic Regression score: %f'
          % logistic_subject.fit(X_train, y_sub).score(X_train, y_sub))

    print('Logistic Regression score: %f'
          % logistic_action.fit(X_train, y_action).score(X_train, y_action))

    print('Logistic Regression score: %f'
          % logistic_area.fit(X_train, y_area).score(X_train, y_area))
    return logistic_subject, logistic_action, logistic_area

In [13]:
X_train, Triplets = process_dataset(d[:1000])
subj_word_to_index,subj_index_to_word,action_word_to_index,action_index_to_word,object_word_to_index,object_index_to_word = process_triplet_dictionary(Triplets)
y_sub_train,y_action_train,y_obj_train = get_y_values(Triplets,subj_word_to_index,action_word_to_index,object_word_to_index)
#f=open('t.csv')

#print_output(y_sub_train, y_action_train, y_obj_train, subj_index_to_word, action_index_to_word, object_index_to_word)

(u'dog', u'running', u'dog')

(u'baby', u'plays', u'croquet')

(u'dog', u'has', u'pink')

(u'dog', u'running', u'beach')

(u'dog', u'standing', u'beach')

(u'cyclist', u'riding', u'pavement')

(u'people', u'watching', u'him')

(u'boy', u'running', u'woodland')

(u'girl', u'in', u'dress')

(u'skier', u'airborne', u'mountains')

(u'photographer', u'looks', u'hills')

(u'bunch', u'of', u'girls')

(u'boat', u'floating', u'waters')

(u'dog', u'catches', u'Frisbee')

(u'lady', u'next_to', u'advertisement')

(u'girl', u'walks', u'trail')

(u'family', u'pose', u'fireplace')

(u'Man', u'in', u'sweater')

(u'dog', u'looks', u'dog')

(u'children', u'in', u'field')

(u'boy', u'raises', u'eyebrow')

(u'dog', u'carrying', u'stick')

(u'dogs', u'racing', u'ractrack')

(u'pitcher', u'throws', u'ball')

(u'dog', u'wades', u'water')

(u'girl', u'takes', u'picture')

(u'jean', u'wearing', u'jacket')

(u'group', u'of', u'people')

(u'front', u'looks', u'camera')

(u'snowboarder', u'jumps', u'high')

(u'gr

In [18]:
X_test = X_train[901:]
y_test_s = y_sub_train[901:]
y_test_a = y_action_train[901:]
y_test_o = y_obj_train[901:]
X_t = (X_train[:901])
y_t_s = y_sub_train[:901]
y_t_a = y_action_train[:901]
y_t_o = y_obj_train[:901]
print(len(X_test),len(y_test_s),len(y_test_a), len(y_test_o))

(88, 88, 88, 88)


In [19]:
#print_output(y_sub_train, y_action_train, y_obj_train, subj_index_to_word, action_index_to_word, object_index_to_word)
lr_s,lr_a,lr_o = train_model(X_t, y_t_s, y_t_a, y_t_o)

Logistic Regression score: 0.319645
Logistic Regression score: 0.128746
Logistic Regression score: 0.100999


In [21]:
y_pred_s = lr_s.predict(X_t)
y_pred_a = lr_a.predict(X_t)
y_pred_o = lr_o.predict(X_t)
print "%s" %lr_s.score(X_t,y_t_s)
print "%s" %lr_a.score(X_t,y_t_a)
print "%s" %lr_o.score(X_t,y_t_o)
print "%s" %lr_s.score(X_test,y_test_s)
print "%s" %lr_a.score(X_test,y_test_a)
print "%s" %lr_o.score(X_test,y_test_o)

0.319644839068
0.128745837958
0.100998890122
0.272727272727
0.0795454545455
0.0227272727273


In [280]:
print_output(y_pred_s, y_pred_a, y_pred_o, subj_index_to_word, action_index_to_word, object_index_to_word)

dog in water
man in ball
dog in field
dog running beach
dog in ball
man in ball
man in water
man in ball
girl in ball
man on mountain
dog in water
man in water
dog in water
dog in ball
man in water
dog in water
dog in ball
dog in water
dog in ball
dog in water
man in water
dog in water
dog in ball
man in ball
dog in water
man in ball
dog in water
man in water
man in water
man on water
man on mountain
dog in field
dog in water
dog in ball
man in rock
man in water
dog in water
man in wall
man in water
man in water
man in ball
dog in ball
man in water
dog in ball
man in water
dog in water
man in water
dog in water
girl in water
dog in ball
man in mountain
dog in ball
dog in water
dog in water
dog in field
man in water
dog in water
man in water
man on mountain
man on rock
dog in ball
dog in ball
man on bench
dog in ball
dog in ball
man in water
man in water
man in water
man in water
man in water
man in water
man in ball
man in water
man in pool
man in water
dog in ball
dog in ball
man in w

In [27]:
def train_svc(X_train, y_sub, y_action, y_area):
    svc_object = SVC()
    svc_action = SVC()
    svc_scene = SVC()

    print('SVC score SUBJECT: %f'
          % svc_object.fit(X_train, y_sub).score(X_train, y_sub))

    print('SVC score ACTION: %f'
          % svc_action.fit(X_train, y_action).score(X_train, y_action))

    print('SVC score AREA: %f'
          % svc_scene.fit(X_train, y_area).score(X_train, y_area))
    return svc_object, svc_action, svc_scene

In [28]:
svc_s,svc_a,svc_o = train_svc(X_t, y_t_s, y_t_a, y_t_o)

SVC score SUBJECT: 0.160932
SVC score ACTION: 0.112098
SVC score AREA: 0.027747


In [ ]:
print "%s" %svc_s.score(X_t,y_t_s)
print "%s" %lr_a.score(X_t,y_t_a)
print "%s" %lr_o.score(X_t,y_t_o)
print "%s" %svc_s.score(X_test,y_test_s)
print "%s" %lr_a.score(X_test,y_test_a)
print "%s" %lr_o.score(X_test,y_test_o)